# CA4

Group 37

Group members:
* Jannicke Ådalen
* Marcus Dalaker Figenschou
* Rikke Sellevold Vegstein

In [ ]:
# Standard library imports
import os

# Third-party imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Scikit-learn imports
from sklearn import ensemble as ens
from sklearn import metrics as met
from sklearn import model_selection as msel
from sklearn import pipeline as pipe

In [ ]:
# Setting the styles of plots so that they have same styling throughout
sns.set_style("white")
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.rcParams["axes.spines.left"] = False
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False

# Set working directory
if "CA4" in os.getcwd():
    os.chdir("..")  # Go up one level if we're in CA3

print(f"Working directory now: {os.getcwd()}")

# Load data
train_path = os.path.join("CA4", "assets", "train.csv")
test_path = os.path.join("CA4", "assets", "test.csv")

# Load data
# 1. Load data
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Data inspection and cleaning

In [ ]:
print("---TRAIN DATA---")
train_df.info()
print("---TEST DATA---")
test_df.info()

## Metadata
A lot of metadata in this dataset (data that isnt physical properties) that is just used to organize the data for the future by astronomers. Seems also that u might be missing values since it has fewer entries and shows up as non-null, will investigate after metadata removal.

In [ ]:
features = ["u", "g", "r", "i", "z", "redshift", "class"]
# Keeping everything except metadata columns
train_df = train_df[features]
print(train_df.head())

# Modifying test_data
test_df = test_df[features[:-1]]
print(test_df.head())

In [ ]:
print("--- Train Data ---")
print(train_df.describe())

print("--- Test Data ---")
print(test_df.describe())

Two things to see here. From the .info we can see that there are fewer observations of u than the rest and we have a min value -9999 for u, g and z which only shows up in train data but not in test data. But in our original data inspection we found no missing values, so this might indicate placeholder.

In [ ]:
print(train_df.isnull().sum())


We can now see that u is actually missing 362 values. Because of our datasize of 80k, we can comfortably remove 362 samples from our dataset of 80k.

In [ ]:
train_df = train_df.dropna()

print(train_df.isnull().sum())

In [ ]:
print("--- Train Data ---")
train_df.info()

print(train_df.describe())

There are still values with -9999, will investigate this further. This might be placeholder values, lets check.

In [ ]:
negative_counts = (train_df.drop(columns=["class"]) == -9999).sum()
print("Negative value counts:")
print(negative_counts)

Yes there is exactly one value with -9999 in the u, g and z columns. Will remove these

In [ ]:
train_df = train_df.replace(-9999, np.nan)

train_df = train_df.dropna()

print(train_df.isnull().sum())

In [ ]:
print(train_df.describe())

In [ ]:
print(train_df["class"].value_counts(normalize=True).mul(100).round(2).astype(str) + " %")

The class distribution is heavily skewed towards galaxies. Roghly 60 % of the dataset are galaxies. We can balance this out by downsampling galaxies, we will do this later just before we split our data into test/val.

# Data plotting

Before we go forward with plotting we will map galaxy, quasars and stars to their given values for submission. We will keep the original labels since this is more descriptive when we plot the data

In [ ]:
class_mapping = {"GALAXY": 0, "QSO": 1, "STAR": 2}
# Keep original text labels for plotting
train_df["class_labels"] = train_df["class"]
# Replaceing with numeric codes
train_df["class"] = train_df["class"].map(class_mapping)

Since there is ~ 79k samples in our dataset we choose to extract a random sample of 5000. This is statistically enought to visualize the data distribution without needing to visualize ~ 79k samples which for some plots takes a lot of time.

In [ ]:
features = train_df.columns.drop(["class", "class_labels"])
classes = train_df["class_labels"].unique()

sample_size = 5000
plot_df = train_df.columns.drop(["class"])
plot_df_sample = train_df.sample(n=sample_size, random_state=42)

## Boxplot

In [ ]:

fig, axes = plt.subplots(3, 2, figsize=(15, 15))
axes = axes.flatten()

for i, feature in enumerate(features):
    sns.boxplot(
        x="class_labels",
        y=feature,
        data=plot_df_sample,
        hue="class_labels",
        palette="Set1",
        ax=axes[i],
    )
    axes[i].set_title(f"Boxplot of {feature}")
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")

    axes[i].tick_params(axis="x", labelsize=14)
    axes[i].patch.set_edgecolor("black")
    axes[i].patch.set_linewidth(1)

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(hspace=0.2)
plt.savefig("boxplot")
plt.show()


Based on the boxplot quasars has the most outliers.

## Violinplot

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(15, 15))
axes = axes.flatten()
for i, feature in enumerate(features):
    sns.violinplot(
        data=plot_df_sample,
        x="class_labels",
        y=feature,
        hue="class_labels",
        palette="Set1",
        ax=axes[i],
        alpha=0.4,
        orient="v",
    )
    axes[i].set_title(f"Violinplot of {feature}")
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")
    axes[i].tick_params(axis="x", labelsize=14)
    axes[i].patch.set_edgecolor("black")
    axes[i].patch.set_linewidth(1)
# Adjust layout
plt.tight_layout()
fig.subplots_adjust(hspace=0.2)
plt.show()

The violinplot shows that galaxy is biomodal for all features. Star is somewhat biomodal for features: g, r and i. Quasars is only unimodal.

It is interesting that galaxies are biomodal for all features, this might be due to galaxies come in different sizes and shapes, and also have varying degrees of luminosity.

That some features show stars as biomodal is also to be expected as stars vary in size and shape, but we did believe that this effect should be stronger than it is .

The quasars are unimodal for all features.

## Feature Correlation Matrix

In [ ]:
samples = [plot_df_sample[plot_df_sample["class_labels"] == cls][features] for cls in classes]

# Set up the figure and axes
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

# Plot correlation matrix for each class
for i, (df, title) in enumerate(zip(samples, classes)):
    correlation_matrix = df.corr()
    sns.heatmap(
        correlation_matrix,
        annot=True,
        cmap="coolwarm",
        fmt=".2f",
        ax=axes[i],
        square=True,
        vmin=-1.0,
        vmax=1.0,
    )
    axes[i].set_title(f"Correlation Matrix - {title} (n={len(df)})")
fig.delaxes(axes[3])
plt.tight_layout()
plt.xticks(rotation=90)
fig.subplots_adjust(hspace=0.2)
plt.show()

From the correlation matrix we can see that redshift will be our most important feature when it comes to seperating the three categories.

# Creating New Features

Now that we have a balanced dataset, we can create new features based on color indices—a method astronomers use to assess the light intensity of celestial objects. We must also remember to add these features to our original training data and test data.

We can group the filters based on thier photometric system like this:

Visible light:
- g(green filter)
- r(red filter)

Ultraviolet light:
- u(ultraviolet)

Infrared spectrum:
- i(near infrared)
- z(infrared)

In [ ]:
# creating a definition we can use later to create new feature datasets


def feature_engineering(df):
    """Adding astronomical color indices to the dataframe."""
    df_features = df.copy()

    df_features["r-z"] = df_features["r"] - df_features["z"]
    df_features["g-r"] = df_features["g"] - df_features["r"]
    df_features["u-r"] = df_features["u"] - df_features["r"]
    df_features["i-z"] = df_features["i"] - df_features["z"]
    df_features["i-r"] = df_features["i"] - df_features["r"]
    df_features["u-g"] = df_features["u"] - df_features["g"]
    df_features["g-i"] = df_features["g"] - df_features["i"]

    # redshift is the feature that seems to seperate the classes best so we will create some more

    df_features["redshift_squared"] = df_features["redshift"] ** 2
    df_features["log_redshift"] = np.log1p(df_features["redshift"])

    # Simple multiplication interactions
    df_features["redshift_u"] = df_features["redshift"] * df_features["u"]

    # We try to create a brightness feature here to
    df_features["spectral_contrast"] = df_features[["u-g", "g-r", "i-r", "i-z"]].max(axis=1) - df_features[
        ["u-g", "g-r", "i-r", "i-z"]
    ].min(axis=1)

    return df_features

In [ ]:
train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)

We ran first a random forest model with all the features above and then we plotted the cumuluative feature importances and chose to keep the features that had ~95% relevance for our model, these are the features we we ended up with.

In [ ]:
top_features = [
    "redshift_squared",
    "log_redshift",
    "redshift",
    "redshift_u",
    "g-i",
    "g-r",
    "spectral_contrast",
    "i-r",
    "r-z",
    "u-r",
    "g",
    "class",
    "class_labels",
]

In [ ]:
train_df_rf = train_df[top_features]
print(train_df_rf)

test_df_rf = test_df[top_features[:-2]]
print(test_df_rf.info())

# Training the models
Since we are working with three categorical target values and a large dataset we have chosen to go for the following models:
- Random Forest Classifier
- Logistic Regression model
- SVC

In [ ]:
X = train_df.drop(columns=["class", "class_labels"])
y = train_df["class"]

# Splitting the data into train and test data with a 60/40 split
X_train, X_test, y_train, y_test = msel.train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

print(X_train)

## Random Forest Classifier

Random forest is especially good for unbalanced data and multi category classification problems.

In [ ]:
random_forest = ens.RandomForestClassifier(n_jobs=-1, random_state=42)

rf_pipe = pipe.Pipeline([("random_forest", random_forest)])


# Parameter distributions for random search
param_grid_rf = {
    "random_forest__n_estimators": [100, 200, 300, 400, 500],
    "random_forest__max_depth": [5, 10, 15, 20, 25, None],
    "random_forest__min_samples_split": [2, 5, 10, 15, 20],
    "random_forest__max_features": ["sqrt", "log2"],
    "random_forest__criterion": ["gini", "entropy", "log_loss"],
}

# Since data is unbalanced we will use stratifiedkfold
stratified_cv = msel.StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# We will use randomizedsearchCV since we are doing hyperparamter tuning
rf_search = msel.RandomizedSearchCV(
    rf_pipe,
    param_distributions=param_grid_rf,
    n_iter=100,
    # Using StratifiedKFold
    cv=stratified_cv,
    n_jobs=-1,
    scoring="f1_macro",
    return_train_score=True,
)

rf_search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % rf_search.best_score_)
print(rf_search.best_params_)


# Print best results
print("Best parameter (CV score=%0.3f):" % rf_search.best_score_)
print(rf_search.best_params_)

In [ ]:
# Evaluate on test set
best_rf_model = rf_search.best_estimator_
y_test_pred_rf = best_rf_model.score(X_test, y_test)
print(f"Test set score with best model: {y_test_pred_rf:.3f}")
print(best_rf_model)

In [ ]:
# Get predictions on the sample test set
y_pred_rf = best_rf_model.predict(X_test)

# Print classification report
print("Classification Report on Sample Test Set:")
print(met.classification_report(y_test, y_pred_rf))

# Create confusion matrix
print("Confusion Matrix on Sample Test Set:")
conf_matrix = met.confusion_matrix(y_test, y_pred_rf)
print(conf_matrix)

# Visualize confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(
    conf_matrix,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=best_rf_model.classes_,
    yticklabels=best_rf_model.classes_,
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (Sample Test Set)")
plt.tight_layout()
plt.show()

In [ ]:
# Extract best parameters from the best model
best_n_estimators = rf_search.best_params_["random_forest__n_estimators"]
best_max_depth = rf_search.best_params_["random_forest__max_depth"]
best_min_samples_split = rf_search.best_params_["random_forest__min_samples_split"]
best_max_features = rf_search.best_params_["random_forest__max_features"]
best_criterion = rf_search.best_params_["random_forest__criterion"]

In [ ]:
print("Training final model on the entire dataset...")

# Prepare full data
X_full = train_df.drop(columns=["class_labels", "class"])
y_full = train_df["class"]


final_rf = ens.RandomForestClassifier(
    n_estimators=best_n_estimators,
    max_features=best_max_features,
    max_depth=best_max_depth,
    criterion=best_criterion,
    random_state=42,
    n_jobs=-1,  # Use all available cores
)


# Create the final pipeline with scaling
final_rf_pipeline = pipe.Pipeline([("random_forest", final_rf)])

# Train the pipeline on unscaled data - pipeline handles scaling internally
final_rf_pipeline.fit(X_full, y_full)

print(f"Final rf model trained on {len(X_full)} samples")

### Prediction

In [ ]:
y_test_rf = final_rf_pipeline.predict(test_df)

# Create the final dataframe with numeric class predictions
y_test_rf = pd.DataFrame(y_test_rf, columns=["class"])
y_test_rf.index.name = "ID"

# Add file path with appropriate naming related to model parameters
base_dir = os.path.join("CA4", "results")
os.makedirs(base_dir, exist_ok=True)
filename = f"random_forest_model.csv"  # noqa: F541
file_path = os.path.join(base_dir, filename)

# Save to CSV
y_test_rf[["class"]].to_csv(file_path)
print(f"Saved rf submission to {file_path}")


def test_long_line():
    very_long_string = "This is an extremely long string that definitely exceeds the 120 character limit that you've set in your Ruff configuration for the Zed editor and should trigger formatting or linting warnings when you save this file or run Ruff manually on it."

    # Here's another long line with a list comprehension that exceeds 120 characters
    very_long_list = [
        x
        for x in range(1000)
        if x % 2 == 0 and x % 3 == 0 and x % 5 == 0 and x % 7 == 0 and x % 11 == 0 and str(x).startswith("1")
    ]

    return very_long_string, very_long_list